# SVM Parameter Optimization Assignment
This notebook performs Support Vector Machine (SVM) parameter optimization on a multi-class dataset. We use the `Wine` dataset from `sklearn.datasets` which contains 3 classes and 13 features.

## Objective:
- Divide the dataset into 10 different samples (70% train, 30% test)
- For each sample, optimize SVM parameters (`kernel`, `nu`, `epsilon`) over 100 iterations
- Record the best parameters and accuracy
- Plot convergence graph for the best sample


In [ ]:
from sklearn.datasets import load_wine
from sklearn.svm import NuSVR
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random

# Load dataset
data = load_wine()
X, y = data.data, data.target

# Parameters and storage
kernels = ['rbf', 'poly', 'sigmoid']
nu_values = np.linspace(0.1, 0.9, 5)
epsilon_values = np.linspace(0.01, 1.0, 5)
results = []
convergence_data = []

# Sample-wise optimization
for sample_id in range(10):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=sample_id)
    best_acc = 0
    best_params = {}
    acc_curve = []
    for _ in range(100):
        kernel = random.choice(kernels)
        nu = random.choice(nu_values)
        epsilon = random.choice(epsilon_values)

        model = make_pipeline(StandardScaler(), NuSVR(kernel=kernel, nu=nu, C=1.0, epsilon=epsilon))
        model.fit(X_train, y_train)
        preds = model.predict(X_test)
        acc = accuracy_score(y_test, np.round(preds).astype(int))
        acc_curve.append(acc)
        if acc > best_acc:
            best_acc = acc
            best_params = {'kernel': kernel, 'nu': nu, 'epsilon': epsilon}
    results.append({
        'Sample': f"S{sample_id+1}",
        'Best Accuracy': round(best_acc * 100, 2),
        'Best Params': best_params
    })
    convergence_data.append(acc_curve)


In [ ]:
# Save results
df = pd.DataFrame(results)
df.to_csv("results_table.csv", index=False)
df


In [ ]:
# Plot convergence for best sample
best_index = np.argmax([r['Best Accuracy'] for r in results])
plt.figure(figsize=(10, 5))
plt.plot(convergence_data[best_index])
plt.title("Convergence Graph of Best SVM")
plt.xlabel("Iteration")
plt.ylabel("Accuracy")
plt.grid()
plt.savefig("convergence_graph.png")
plt.show()
